# WeatherPy

---

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

**Discussion about the linear relationship:** YOUR RESPONSE HERE

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import sys

# Add the directory containing citypy.py to the system path
sys.path.append('/Users/syedshahid/python-api-challenge')  
import citypy

# Import the OpenWeatherMap API key
weather_api_key = "803fbd5e4e89b023bc38d8cac4fe6bbe"

# Function to retrieve weather data for a city
def get_weather_data(city):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": weather_api_key,
        "units": "metric"  # Use metric units for temperature in Celsius
    }
    response = requests.get(base_url, params=params)
    return response.json()

# Function to extract relevant weather information from the API response
def extract_weather_info(response):
    try:
        city_name = response['name']
        country_code = response['sys']['country']
        latitude = response['coord']['lat']
        longitude = response['coord']['lon']
        temperature = response['main']['temp']
        humidity = response['main']['humidity']
        cloudiness = response['clouds']['all']
        wind_speed = response['wind']['speed']
        return {
            'City': city_name,
            'Country': country_code,
            'Latitude': latitude,
            'Longitude': longitude,
            'Temperature (C)': temperature,
            'Humidity (%)': humidity,
            'Cloudiness (%)': cloudiness,
            'Wind Speed (m/s)': wind_speed
        }
    except KeyError:
        print(f"Missing data for {response['name']}. Skipping...")
        return None

# Generate the list of cities
lat_lngs = []

# Empty list for holding the city names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citypy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to the cities list
    if city not in cities:
        cities.append(city)

# Print the number of cities in the list
print(f"Number of cities in the list: {len(cities)}")

# Create an empty list to hold the weather data for each city
weather_data = []

# Loop through each city in the list and retrieve weather data
for city in cities:
    response = get_weather_data(city)
    if response['cod'] == 200:
        weather_info = extract_weather_info(response)
        if weather_info:
            weather_data.append(weather_info)
    else:
        print(f"City '{city}' not found. Skipping...")

# Convert the weather data list into a pandas DataFrame
weather_df = pd.DataFrame(weather_data)

# Display the DataFrame
weather_df.head()

# Create scatter plots to showcase the relationship between weather variables and latitude
def plot_weather_relationship(x, y, hemisphere, xlabel, ylabel):
    plt.scatter(x, y, marker='o', edgecolors='black', alpha=0.75)
    plt.title(f"{hemisphere} Hemisphere: {xlabel} vs. {ylabel}")
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.show()

# Latitude vs. Temperature
plot_weather_relationship(weather_df['Latitude'], weather_df['Temperature (C)'], 'Latitude', 'Latitude', 'Temperature (C)')

# Latitude vs. Humidity
plot_weather_relationship(weather_df['Latitude'], weather_df['Humidity (%)'], 'Latitude', 'Latitude', 'Humidity (%)')

# Latitude vs. Cloudiness
plot_weather_relationship(weather_df['Latitude'], weather_df['Cloudiness (%)'], 'Latitude', 'Latitude', 'Cloudiness (%)')

# Latitude vs. Wind Speed
plot_weather_relationship(weather_df['Latitude'], weather_df['Wind Speed (m/s)'], 'Latitude', 'Latitude', 'Wind Speed (m/s)')

# Compute linear regression for each relationship and plot the regression lines
def plot_linear_regression(x, y, xlabel, ylabel):
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
    regress_values = x * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(x, y)
    plt.plot(x, regress_values, "r-")
    plt.annotate(line_eq, (min(x), min(y)), fontsize=12, color="red")
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

# Northern Hemisphere: Temperature vs. Latitude
northern_hemi_df = weather_df.loc[weather_df['Latitude'] >= 0]
plot_linear_regression(northern_hemi_df['Latitude'], northern_hemi_df['Temperature (C)'], 'Latitude', 'Temperature (C)')

# Southern Hemisphere: Temperature vs. Latitude
southern_hemi_df = weather_df.loc[weather_df['Latitude'] < 0]
plot_linear_regression(southern_hemi_df['Latitude'], southern_hemi_df['Temperature (C)'], 'Latitude', 'Temperature (C)')


ModuleNotFoundError: No module named 'citypy'